In [26]:
import pandas as pd
from pydrake.all import PiecewisePolynomial

def make_wrist_trajectories(csv_path: str,
                             left_id: int = 15,
                             right_id: int = 16):
    df = pd.read_csv(csv_path)

    left = df[df["landmark_id"] == left_id].copy()
    right = df[df["landmark_id"] == right_id].copy()

    left = left.sort_values("time_sec")
    right = right.sort_values("time_sec")

    merged = pd.merge(
        left[["time_sec", "x", "y", "z"]],
        right[["time_sec", "x", "y", "z"]],
        on="time_sec",
        suffixes=("_L", "_R")
    ).sort_values("time_sec")

    t = merged["time_sec"].to_numpy()
    t0 = t[0]
    breaks = t - t0  # start at 0

    left_samples = merged[["x_L", "y_L", "z_L"]].to_numpy().T   # (3, N)
    right_samples = merged[["x_R", "y_R", "z_R"]].to_numpy().T  # (3, N)

    left_traj = PiecewisePolynomial.FirstOrderHold(breaks, left_samples)
    right_traj = PiecewisePolynomial.FirstOrderHold(breaks, right_samples)
    return left_traj, right_traj

csv_path = "videos/keypoints_clip.csv"  # adjust if needed
left_traj, right_traj = make_wrist_trajectories(csv_path)

print("Traj time:", left_traj.start_time(), "→", left_traj.end_time())


Traj time: 0.0 → 3.0


In [27]:
from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder, Parser,
    SpatialInertia, UnitInertia, RigidTransform,
    PrismaticJoint, WeldJoint
)
import numpy as np

# Build diagram pieces
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant)

# Inertia for the little "stage" bodies that carry the hands
mass = 0.1
inertia = UnitInertia.SolidBox(0.05, 0.05, 0.05)
M = SpatialInertia(mass=mass,
                   p_PScm_E=np.zeros(3),
                   G_SP_E=inertia)

# Model instances for the stages
left_stage = plant.AddModelInstance("left_hand_stage")
right_stage = plant.AddModelInstance("right_hand_stage")

# ================= LEFT HAND =================
# Stage bodies
left_x_body = plant.AddRigidBody("left_x_body", left_stage, M)
left_y_body = plant.AddRigidBody("left_y_body", left_stage, M)
left_z_body = plant.AddRigidBody("left_z_body", left_stage, M)

# Prismatic joints (world -> x -> y -> z)
left_jx = plant.AddJoint(
    PrismaticJoint("left_jx",
                   plant.world_frame(),
                   left_x_body.body_frame(),
                   np.array([1., 0., 0.]))    # X axis
)
left_jy = plant.AddJoint(
    PrismaticJoint("left_jy",
                   left_x_body.body_frame(),
                   left_y_body.body_frame(),
                   np.array([0., 1., 0.]))    # Y axis
)
left_jz = plant.AddJoint(
    PrismaticJoint("left_jz",
                   left_y_body.body_frame(),
                   left_z_body.body_frame(),
                   np.array([0., 0., 1.]))    # Z axis
)

# Default starting position
left_jx.set_default_translation(-0.35)
left_jy.set_default_translation(0.0)
left_jz.set_default_translation(0.9)

# Joint actuators (these are your "hand actuators")
left_ax = plant.AddJointActuator("left_ax", left_jx)
left_ay = plant.AddJointActuator("left_ay", left_jy)
left_az = plant.AddJointActuator("left_az", left_jz)

# Load left hand SDF and weld it to the end of the stage
left_model = parser.AddModels("assets/left_hand.sdf")[0]
left_body_index = plant.GetBodyIndices(left_model)[0]
left_sdf_body = plant.get_body(left_body_index)

plant.AddJoint(
    WeldJoint("left_hand_weld",
              left_z_body.body_frame(),
              left_sdf_body.body_frame(),
              RigidTransform())
)

# ================= RIGHT HAND =================
# Stage bodies
right_x_body = plant.AddRigidBody("right_x_body", right_stage, M)
right_y_body = plant.AddRigidBody("right_y_body", right_stage, M)
right_z_body = plant.AddRigidBody("right_z_body", right_stage, M)

# Prismatic joints
right_jx = plant.AddJoint(
    PrismaticJoint("right_jx",
                   plant.world_frame(),
                   right_x_body.body_frame(),
                   np.array([1., 0., 0.]))
)
right_jy = plant.AddJoint(
    PrismaticJoint("right_jy",
                   right_x_body.body_frame(),
                   right_y_body.body_frame(),
                   np.array([0., 1., 0.]))
)
right_jz = plant.AddJoint(
    PrismaticJoint("right_jz",
                   right_y_body.body_frame(),
                   right_z_body.body_frame(),
                   np.array([0., 0., 1.]))
)

right_jx.set_default_translation(0.35)
right_jy.set_default_translation(0.0)
right_jz.set_default_translation(0.9)

right_ax = plant.AddJointActuator("right_ax", right_jx)
right_ay = plant.AddJointActuator("right_ay", right_jy)
right_az = plant.AddJointActuator("right_az", right_jz)

right_model = parser.AddModels("assets/right_hand.sdf")[0]
right_body_index = plant.GetBodyIndices(right_model)[0]
right_sdf_body = plant.get_body(right_body_index)

plant.AddJoint(
    WeldJoint("right_hand_weld",
              right_z_body.body_frame(),
              right_sdf_body.body_frame(),
              RigidTransform())
)

# Finalize the plant (no more bodies/joints after this)
plant.Finalize()


In [28]:
from pydrake.systems.framework import LeafSystem, BasicVector
import numpy as np

class SimpleHandPd(LeafSystem):
    """
    Minimal PD controller for two hands.
    Inputs:
      - q_ref_left:  3x1 desired [x, y, z] for left hand
      - q_ref_right: 3x1 desired [x, y, z] for right hand
      - x:           plant state [q; v]
    Output:
      - u:           full actuator vector (only hand actuators nonzero)
    """
    def __init__(self, plant,
                 left_joints, right_joints,
                 left_acts, right_acts,
                 kp=800.0, kd=40.0):
        super().__init__()
        self.plant = plant

        self.left_joints = left_joints
        self.right_joints = right_joints
        self.left_acts = left_acts
        self.right_acts = right_acts

        # Indices in q and v
        self.left_q_idx = np.array([j.position_start() for j in left_joints], dtype=int)
        self.right_q_idx = np.array([j.position_start() for j in right_joints], dtype=int)
        self.left_v_idx = np.array([j.velocity_start() for j in left_joints], dtype=int)
        self.right_v_idx = np.array([j.velocity_start() for j in right_joints], dtype=int)

        # Indices in actuation vector
        self.left_u_idx = np.array([a.index() for a in left_acts], dtype=int)
        self.right_u_idx = np.array([a.index() for a in right_acts], dtype=int)

        self.kp = float(kp)
        self.kd = float(kd)

        # Input ports
        self.q_ref_left_input = self.DeclareVectorInputPort("q_ref_left", BasicVector(3))
        self.q_ref_right_input = self.DeclareVectorInputPort("q_ref_right", BasicVector(3))
        self.state_input = self.DeclareVectorInputPort("x", BasicVector(plant.num_multibody_states()))

        # Output: all actuator efforts
        self.DeclareVectorOutputPort(
            "u",
            BasicVector(plant.num_actuators()),
            self.CalcOutput
        )

    def CalcOutput(self, context, output):
        q_ref_left = self.q_ref_left_input.Eval(context)
        q_ref_right = self.q_ref_right_input.Eval(context)
        x = self.state_input.Eval(context)

        nq = self.plant.num_positions()
        q = x[:nq]
        v = x[nq:]

        q_left = q[self.left_q_idx]
        v_left = v[self.left_v_idx]
        q_right = q[self.right_q_idx]
        v_right = v[self.right_v_idx]

        u_left = self.kp * (q_ref_left - q_left) - self.kd * v_left
        u_right = self.kp * (q_ref_right - q_right) - self.kd * v_right

        u = np.zeros(self.plant.num_actuators())
        u[self.left_u_idx] = u_left
        u[self.right_u_idx] = u_right

        output.SetFromVector(u)


In [29]:
from pydrake.systems.primitives import TrajectorySource
from pydrake.all import StartMeshcat, MeshcatVisualizer
from pydrake.systems.analysis import Simulator

# 1) Trajectory sources (desired [x,y,z] for each hand)
left_traj_src = builder.AddSystem(TrajectorySource(left_traj))
right_traj_src = builder.AddSystem(TrajectorySource(right_traj))

# 2) PD controller system
left_joints = (left_jx, left_jy, left_jz)
right_joints = (right_jx, right_jy, right_jz)
left_acts = (left_ax, left_ay, left_az)
right_acts = (right_ax, right_ay, right_az)

pd_ctrl = builder.AddSystem(
    SimpleHandPd(
        plant,
        left_joints=left_joints,
        right_joints=right_joints,
        left_acts=left_acts,
        right_acts=right_acts,
        kp=800.0,
        kd=40.0
    )
)

# Connect desired positions
builder.Connect(left_traj_src.get_output_port(),
                pd_ctrl.GetInputPort("q_ref_left"))
builder.Connect(right_traj_src.get_output_port(),
                pd_ctrl.GetInputPort("q_ref_right"))

# Connect plant state and actuation
builder.Connect(plant.get_state_output_port(),
                pd_ctrl.GetInputPort("x"))
builder.Connect(pd_ctrl.get_output_port(),
                plant.get_actuation_input_port())

# 3) Meshcat visualization
meshcat = StartMeshcat()
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

# 4) Build diagram and simulate
diagram = builder.Build()
context = diagram.CreateDefaultContext()

sim = Simulator(diagram, context)
sim.set_target_realtime_rate(1.0)

tf = min(left_traj.end_time(), right_traj.end_time())
sim.AdvanceTo(tf)


INFO:drake:Meshcat listening for connections at http://localhost:7002
